# 3D sz's MODEL

- configuration A: (128 x 128 x 64 nodes) 128 CPUS
- configuration B: (256 x 256 x 96 nodes) 256 CPUS
The user should remove visualisation from the python script before running the model on raijin.
In [2]:


In [37]:
import matplotlib.pyplot as plt
import UWGeodynamics as GEO
u = GEO.UnitRegistry # u. +tab to see available units or u = GEO.u
import glucifer

In [38]:
## Characteristic values of the system

half_rate = 1.8 * u.centimeter / u.year # What is it?
model_length = 10000e3 * u.meter
model_width = 2000e3 * u.meter
surfaceTemp = 273.15 * u.degK
baseModelTemp = 3570 * u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2 # what

KL = model_length # Characteristic length
Kt = KL / half_rate # Characteristic time 
KM = bodyforce * KL**2 * Kt**2 # Characteristic mass
KT = (baseModelTemp - surfaceTemp) # Characteristic temperature

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM
GEO.scaling_coefficients["[temperature]"] = KT


In [39]:
## definition of central object: define the geometry of the problem: container for the field variables

Model = GEO.Model(elementRes=(32, 32, 16), 
                  minCoord=(0. * u.kilometer, -0. * u.kilometer, -2880. * u.kilometer), 
                  maxCoord=(10000. * u.kilometer, 2000. * u.kilometer, 20. * u.kilometer), 
                  gravity=(0.0 ,0.0, -9.81 * u.meter / u.second**2))

# Fig = glucifer.Figure()
# Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=1)
# viewer = Fig.viewer(resolution=(600,300))
# viewer.window()


## Output

In [40]:
Model.outputDir="3D"

## Define Materials
The model has initially 4 materials (air, crust, mantle lithosphere and mantle). We add a fifth material for the sediment. Sediment will only appear if surface processes are turned on...(and if there is sedimentation of course)

In [42]:
## Definition of materials

air               = Model.add_material(name="Air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=2.0 * u.kilometer))
stickyAir         = Model.add_material(name="StickyAir", shape=GEO.shapes.Layer3D(top=air.bottom, bottom= 0.0 * u.kilometer))
uppercrust        = Model.add_material(name="UppperCrust", shape=GEO.shapes.Layer3D(top=stickyAir.bottom, bottom=-15.0 * u.kilometer))
midcrust          = Model.add_material(name="MidCrust", shape=GEO.shapes.Layer3D(top=uppercrust.bottom * u.kilometer, bottom=-20.0 * u.kilometer))
lowercrust        = Model.add_material(name="LowerCrust", shape=GEO.shapes.Layer3D(top=midcrust.bottom * u.kilometer, bottom=-40.0 * u.kilometer))
mantleLithosphere = Model.add_material(name="MantleLithosphere", shape=GEO.shapes.Layer3D(top=lowercrust.bottom * u.kilometer, bottom=-120.0 * u.kilometer))
#asthenosphere     = Model.add_material(name="Asthenosphere", shape=GEO.shapes.Layer3D(top=mantleLithosphere.bottom * u.kilometer, bottom=-670.0 * u.kilometer))
mantle            = Model.add_material(name="Mantle", shape=GEO.shapes.Layer3D(top=mantleLithosphere.bottom, bottom=Model.bottom))
#sediment          = Model.add_material(name="Sediment")


## Definition of slab shape with 1 layer and 1 halfspace (print on previous layers)

slabLayer = GEO.shapes.Layer3D(top=-0.*u.kilometer, bottom=-2000.*u.kilometer)
slopeSlab = GEO.shapes.HalfSpace(normal=(-1.,0.,1.), origin=(5000.*u.kilometer,1000.*u.kilometer,-2000.*u.kilometer))
#rightWall= GEO.shapes.HalfSpace(normal=(1.0,0.,0.), origin=(10000.*u.kilometer,0.*u.kilometer,-1000.*u.kilometer))


CompositeShape = slopeSlab & slabLayer #& rightWall

Slab = Model.add_material(name="Slab", shape=CompositeShape)




Fig = glucifer.Figure()
Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=0.70)
viewer = Fig.viewer(resolution=(1200,600))
viewer = Fig.viewer(axis=True)
viewer.window()

Model_MaterialField=(Model.projMaterialField)


<IPython.core.display.Javascript object>

In [43]:
# Model.surfaceProcesses = GEO.surfaceProcesses.Badlands(airIndex=[air.index], sedimentIndex=sediment.index,
#                                            XML="ressources/badlands.xml", resolution=1. * u.kilometer, 
#                                            checkpoint_interval=0.01 * u.megayears)

In [44]:

# test= (Model.projMaterialField)

# plt.close()

# plt.imshow(test.data[:], interpolation='nearest', origin='lower',
#           extent=[Model.minCoord[0].magnitude,Model.maxCoord[0].magnitude, 
#                   Model.minCoord[2].magnitude, Model.maxCoord[2].magnitude], alpha=1, rasterized=True,cmap='RdYlBu_r') # this plots the whole materials.


# Model = GEO.Model(elementRes=(32, 32, 16), 
#                   minCoord=(0. * u.kilometer, -0. * u.kilometer, -2880. * u.kilometer), 
#                   maxCoord=(10000. * u.kilometer, 2000. * u.kilometer, 20. * u.kilometer), 
#                   gravity=(0.0 ,0.0, -9.81 * u.meter / u.second**2))


# plt.colorbar()
# axes=plt.gca()
# plt.yticks([-2000,-670,-40,0,20])
# axes.set_xlabel('Distance (km)')
# axes.set_ylabel('Depth (km)')
# axes.set_title('Material Field')
# axes.figure.dpi=300 # 

# plt.show()

## Limiters


In [45]:
Model.stressLimiter = 300.0 * u.megapascal
Model.maxViscosity = 5.0e23 * u.pascal * u.second
Model.minViscosity = 1.0e19 * u.pascal * u.second
Model.diffusivity = 1.0e-6 * u.metre**2 / u.second 
Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

### Material specific definitions


In [56]:
air.diffusivity = 1.0e-6 * u.metre**2 / u.second
stickyAir.diffusivity = 1.0e-6 * u.metre**2 / u.second

air.capacity = 100. * u.joule / (u.kelvin * u.kilogram)
stickyAir.capacity = 100. * u.joule / (u.kelvin * u.kilogram)


uppercrust.diffusivity = 1.1e-6 * u.metre**2 / u.second
midcrust.diffusivity   = 1.1e-6 * u.metre**2 / u.second
lowercrust.diffusivity = 1.1e-6 * u.metre**2 / u.second
mantle.diffusivity = 1.1e-6 * u.metre**2 / u.second
Slab.diffusivity = 1.1e-6 * u.metre**2 / u.second
mantleLithosphere.diffusivity = 1.1e-6 * u.metre**2 / u.second





Model.diffusivity = 1.0e-6 * u.metre**2 / u.second 
Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

In [57]:
air.density               = 1. * u.kilogram / u.metre**3
stickyAir.density         = 1. * u.kilogram / u.metre**3
#sediment.density          = GEO.LinearDensity(reference_density=2200. * u.kilogram / u.metre**3)
uppercrust.density        = GEO.LinearDensity(2400. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
midcrust.density          = GEO.LinearDensity(2620. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
lowercrust.density        = GEO.LinearDensity(2720. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
mantleLithosphere.density = GEO.LinearDensity(3370. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
#asthenosphere.density     = GEO.LinearDensity(3395. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
mantle.density            = GEO.LinearDensity(3395. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
Slab.density            = GEO.LinearDensity(1300. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)

In [58]:
#sediment.radiogenicHeatProd   = 0.60 * u.microwatt / u.meter**3 # faccenda 2008: 1-5
uppercrust.radiogenicHeatProd = 0.70 * u.microwatt / u.meter**3 # faccenda 2008: 1-5
midcrust.radiogenicHeatProd   = 0.40 * u.microwatt / u.meter**3 # faccenda 2008: 1-5
lowercrust.radiogenicHeatProd = 0.30 * u.microwatt / u.meter**3 # faccenda 2008: 0.25, ocenic crust:0,25
mantleLithosphere.radiogenicHeatProd = 0.01358 * u.microwatt / u.meter**3 # faccenda 2008: 0.022
#asthenosphere.radiogenicHeatProd = 0.02e-6 * u.microwatt / u.meter**3

### Viscous Rheologies

In [59]:
rh = GEO.ViscousCreepRegistry() 

#viscosity = GEO.ViscousCreep()
#viscosity.activationEnergy = 200. * u.kilojoule

air.viscosity                = 1e18 * u.pascal * u.second

stickyAir.viscosity          = 1e19 * u.pascal * u.second

# sediment.viscosity           = rh.Kronenberg_et_al_1984 # rh.Gleason_and_Tullis_1995
# sediment.minViscosity        = 1e18 * u.pascal * u.second
# sediment.maxViscosity        = 5e23 * u.pascal * u.second

uppercrust.viscosity    = 1.0 * rh.Goetze_et_al_1978
uppercrust.minViscosity = 1e18 * u.pascal * u.second
uppercrust.maxViscosity = 5e23 * u.pascal * u.second

midcrust.viscosity    = 1.0 * rh.Gleason_and_Tullis_1995
midcrust.minViscosity = 1e18 * u.pascal * u.second
midcrust.maxViscosity = 5e23 * u.pascal * u.second

lowercrust.viscosity    = 1.0 * rh.Wang_et_al_2012
lowercrust.minViscosity = 1e18 * u.pascal * u.second
lowercrust.maxViscosity = 5e23 * u.pascal * u.second

mantleLithosphere.viscosity    = 1.0 * rh.Brace_and_Kohlstedt_1980 # 5.0 * rh.Karato_and_Wu_1990
mantleLithosphere.minViscosity = 1e18 * u.pascal * u.second
mantleLithosphere.maxViscosity = 5e23 * u.pascal * u.second



Slab.viscosity    = 1.0 * rh.Brace_and_Kohlstedt_1980 # 5.0 * rh.Karato_and_Wu_1990
Slab.minViscosity = 1e18 * u.pascal * u.second
Slab.maxViscosity = 5e23 * u.pascal * u.second

# asthenosphere.viscosity    = rh.Brace_and_Kohlstedt_1980 
# asthenosphere.minViscosity = 1e18 * u.pascal * u.second
# asthenosphere.maxViscosity = 5e23 * u.pascal * u.second

mantle.viscosity    = rh.Brace_and_Kohlstedt_1980 # 1.0 * rh.Karato_and_Wu_1990
mantle.minViscosity = 1e18 * u.pascal * u.second
mantle.maxViscosity = 5e23 * u.pascal * u.second

minimumviscosity   = GEO.ConstantViscosity(1e18 * u.pascal * u.second)
maximumviscosity   = GEO.ConstantViscosity(5e23 * u.pascal * u.second)

# rh.Gleason_and_Tullis_1995


## Plasticities

In [60]:
pl = GEO.PlasticityRegistry()

In [61]:
# sediment.plasticity     = pl.Rey_et_al_2010_UpperCrust
# sediment.plasticity.frictionCoefficient = 0.268
# sediment.plasticity.frictionAfterSoftening = 0.01
# sediment.plasticity.epsilon2 = 0.10
# sediment.stressLimiter  = 20 * u.megapascal

uppercrust.plasticity     = pl.Rey_et_al_2010_UpperCrust
uppercrust.plasticity.frictionCoefficient = 0.268
uppercrust.plasticity.frictionAfterSoftening = 0.03
uppercrust.plasticity.epsilon2 = 0.15
uppercrust.stressLimiter  = 100 * u.megapascal

midcrust.plasticity     = pl.Rey_et_al_2010_UpperCrust
midcrust.plasticity.frictionCoefficient = 0.268
midcrust.plasticity.frictionAfterSoftening = 0.07
midcrust.stressLimiter  = 120 * u.megapascal

lowercrust.plasticity     = pl.Rey_et_al_2010_LowerCrust
lowercrust.stressLimiter  = 150 * u.megapascal

mantleLithosphere.plasticity     = pl.Rey_et_al_2010_Mantle
mantleLithosphere.stressLimiter  = 300 * u.megapascal

Slab.plasticity     = pl.Rey_et_al_2010_Mantle
Slab.stressLimiter  = 300 * u.megapascal

# asthenosphere.plasticity     = pl.Rey_et_al_2010_Mantle
# asthenosphere.stressLimiter  = 300 * u.megapascal

mantle.plasticity             = pl.Rey_et_al_2010_Mantle

# or

# mantle.plasticity = GEO.DruckerPrager(
#    cohesion=20.*u.megapascal,
#    cohesionAfterSoftening=20.*u.megapascal,
#    frictionCoefficient=0.12,
#    frictionAfterSoftening=0.02,
#    epsilon1=0.01, epsilon2=1.0)


#pl.Rey_et_al_2010_UpperCrust


In [62]:
#uppercrust

In [63]:
#uppercrust.plasticity

## Partial Melting

In [64]:
solidii = GEO.SolidusRegistry()
crust_solidus = solidii.Crustal_Solidus
mantle_solidus = solidii.Mantle_Solidus


liquidii = GEO.LiquidusRegistry()
crust_liquidus = liquidii.Crustal_Liquidus
mantle_liquidus = liquidii.Mantle_Liquidus


# In our experiments, the viscosity decreases linearly by 3 orders of magnitude 
#when the melt fraction increases from 15 to 30%. When the melt fraction is 15%, 
#the viscosity of the melted crust is that of the non-melted surrounding; 
#when the melt fraction is 30%, its viscosity is a thousand times lower than in surrounding material. 
#Rosenberg and Handy (2005) showed that significant weakening occurs at 7% melt fraction.

uppercrust.add_melt_modifier(crust_solidus, crust_liquidus, 
                         latentHeatFusion=250.0 * u.kilojoules / u.kilogram / u.kelvin,
                         meltFraction=0.,
                         meltFractionLimit=0.3,
                         meltExpansion=0.13, 
                         viscosityChangeX1 = 0.15,
                         viscosityChangeX2 = 0.30,
                         viscosityChange = -1.0e3
                        )
midcrust.add_melt_modifier(crust_solidus, crust_liquidus, 
                         latentHeatFusion=250.0 * u.kilojoules / u.kilogram / u.kelvin,
                         meltFraction=0.,
                         meltFractionLimit=0.3,
                         meltExpansion=0.13, 
                         viscosityChangeX1 = 0.15,
                         viscosityChangeX2 = 0.30,
                         viscosityChange = -1.0e3
                        ) 
lowercrust.add_melt_modifier(crust_solidus, crust_liquidus, 
                         latentHeatFusion=250.0 * u.kilojoules / u.kilogram / u.kelvin,
                         meltFraction=0.,
                         meltFractionLimit=0.3,
                         meltExpansion=0.13, 
                         viscosityChangeX1 = 0.15,
                         viscosityChangeX2 = 0.30,
                         viscosityChange = -1.0e3
                        ) 

mantleLithosphere.add_melt_modifier(mantle_solidus, mantle_liquidus, 
                         latentHeatFusion=450.0 * u.kilojoules / u.kilogram / u.kelvin,
                         meltFraction=0.,
                         meltFractionLimit=0.08,
                         meltExpansion=0.16, 
                         viscosityChangeX1 = 0.00,
                         viscosityChangeX2 = 0.08,
                         viscosityChange = -1.0e2
                        ) 

# asthenosphere.add_melt_modifier(mantle_solidus, mantle_liquidus, 
#                          latentHeatFusion=450.0 * u.kilojoules / u.kilogram / u.kelvin,
#                          meltFraction=0.,
#                          meltFractionLimit=0.08,
#                          meltExpansion=0.16, 
#                          viscosityChangeX1 = 0.00,
#                          viscosityChangeX2 = 0.08,
#                          viscosityChange = -1.0e2
#                         ) 

mantle.add_melt_modifier(mantle_solidus, mantle_liquidus, 
                         latentHeatFusion=450.0 * u.kilojoules / u.kilogram / u.kelvin,
                         meltFraction=0.,
                         meltFractionLimit=0.08,
                         meltExpansion=0.16, 
                         viscosityChangeX1 = 0.00,
                         viscosityChangeX2 = 0.08,
                         viscosityChange = -1.0e2
                        ) 


## Add Temperature Boundary Conditions

Temperature is 293.15K at the top and 3570K at the bottom. Temperature is constant in the mantle and the air layers. 

In [65]:
Model.set_temperatureBCs(top=293.15 * u.degK, 
                         bottom=3570. * u.degK, 
                         indexSets=[(mantle.indices, 1603.15 * u.degK),
                                    (stickyAir.indices, 293.15 * u.degK ),
                                    (air.indices, 293.15 * u.degK )]) 

#materials=[(air, 293.15 * u.degK),(stickyAir, 293.15 * u.degK), (asthenosphere.indices, 1603.15 * u.degK), (mantle.indices, 3570 * u.degK)])  #

## Add Velocity Boundary Conditions

We push on the right side. The back and front wall are freeslip. We use a pseudo isostatic support condition at the bottom.

In [66]:
Model.set_velocityBCs(left=[0. * u.centimeter / u.year, None, None],
                      right=[-2.8 * u.centimeter / u.year, None, None],
                      back=[None, 0., None],
                      front=[None, 0., None],
                      bottom=GEO.LecodeIsostasy(reference_mat=mantle.index, average=False))



## Add Initial Damage with Plastic Strain
An ellipsoidal shape with random damage is used to seed plastic deformation.

In [67]:
import numpy as np

def gaussian(xx, centre, width):
    return ( np.exp( -(xx - centre)**2 / width ))

# maxDamage = 0.7
# Model.plasticStrain.data[:] = maxDamage * np.random.rand(*Model.plasticStrain.data.shape[:])
# Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,0], (GEO.nd(Model.maxCoord[0] - Model.minCoord[0])) / 2.0, GEO.nd(100.0 * u.kilometer))
# Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,1], GEO.nd(-5. * u.kilometer) , GEO.nd(1.0 * u.kilometer))

# maxDamage = 0.5
# Model.plasticStrain.data[:] = maxDamage * np.random.rand(*Model.plasticStrain.data.shape[:])
# Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,0], (GEO.nd(Model.maxCoord[0] - Model.minCoord[0])) / 2.0, GEO.nd(10.0 * u.kilometer))
# Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,1], GEO.nd(-10. * u.kilometer) , GEO.nd(2.0 * u.kilometer))

maxDamage = 0.25
Model.plasticStrain.data[:] = maxDamage * np.random.rand(*Model.plasticStrain.data.shape[:])
Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,0], (GEO.nd(Model.maxCoord[0] - Model.minCoord[0])) / 2.0, GEO.nd(5.0 * u.kilometer))
Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,1], (GEO.nd(Model.maxCoord[1] - Model.minCoord[1])) / 2.0, GEO.nd(10.0 * u.kilometer))
Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,2], GEO.nd(-40. * u.kilometer) , GEO.nd(5.0 * u.kilometer))

# maxDamage = 0.5
# Model.plasticStrain.data[:] = maxDamage * np.random.rand(*Model.plasticStrain.data.shape[:])
# Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,0], (GEO.nd(Model.maxCoord[0] - Model.minCoord[0])) / 2.0, GEO.nd(5.0 * u.kilometer))
# Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,1], (GEO.nd(Model.maxCoord[1] - Model.minCoord[1])) / 2.0, GEO.nd(10.0 * u.kilometer))
# Model.plasticStrain.data[:,0] *= gaussian(Model.swarm.particleCoordinates.data[:,2], GEO.nd(-40. * u.kilometer) , GEO.nd(5.0 * u.kilometer))

#Fig = Model.plot.plastic_strain(script=["rotate z 30", "rotate x -60"], projected=True, figsize=(900,600))

# Plastic Strain Enveloppe

This is to create a buffer zone close to the left and right boundaries where plastic strain is not allowed to accumulate. You can comment the following cell if you think it is not needed.

In [68]:
import underworld.function as fn

def post_hook():
    coords = fn.input()
    zz = coords[0] / (GEO.nd(Model.maxCoord[0]) - GEO.nd(Model.minCoord[0]))
    fact = fn.math.pow(fn.math.tanh(zz*20.0) + fn.math.tanh((1.0-zz)*20.0) - fn.math.tanh(20.0), 4)
    Model.plasticStrain.data[:] = Model.plasticStrain.data[:] * fact.evaluate(Model.swarm)

Model.postSolveHook = post_hook

# Add Passive Tracers 

In [69]:
#help(Model.add_passive_tracers)

In [70]:
import numpy as np

xp = np.linspace(GEO.nd(Model.minCoord[0]), GEO.nd(Model.maxCoord[0]), 100)
yp = np.linspace(GEO.nd(Model.minCoord[1]), GEO.nd(Model.maxCoord[1]), 100)

xp, yp = np.meshgrid(xp, yp)
xp = xp.flatten()
yp = yp.flatten()
zp = np.zeros(xp.shape)

surface_tracers = Model.add_passive_tracers(name="Surface2", vertices=[xp, yp, zp])
moho_tracers = Model.add_passive_tracers(name="Moho", vertices=[xp, yp, zp+GEO.nd(mantleLithosphere.top)])
#moho_tracers = Model.add_passive_tracers(name="Moho", vertices=[x,y-40.*u.kilometer])
#moho_tracers.add_tracked_field(Model.pressureField, name="Pressure" , dataType="float", units=u.megapascal, count=1, overwrite=True)
#surface_tracers.add_tracked_field(Model.pressureField, name="Pressure" , dataType="float", units=u.megapascal, count=1, overwrite=True)

#print moho_tracers
#print surface_tracers

import numpy as np

x2 = np.linspace(Model.minCoord[0], Model.maxCoord[0], 1000) * u.kilometer
y2 = np.linspace(Model.minCoord[0], Model.maxCoord[0], 1000) * u.kilometer
x2, y2 = np.meshgrid(xp, yp)
x2 = x2.flatten()
y2 = y2.flatten()
z2 = -80. * u.kilometer

level_tracers = Model.add_passive_tracers(name="Level", vertices=[x2,y2,z2])
level_tracers2 = Model.level_tracers.add_tracked_field(Model.strainRateField, name="Lev_StrainRate", dataType="float", units=u.sec, count=1, overwrite=True)

# Add ellipses

x_c, y_c, z_c = GEO.circles_grid(radius=2.0*u.kilometer, 
                    minCoord=[0. * u.kilometer, 0. * u.kilometer, lowercrust.bottom], 
                    maxCoord=[10000. * u.kilometer, 2000. * u.kilometer, uppercrust.top])
circles_c = Model.add_passive_tracers(name="FSE_Crust", vertices=[x_c, y_c, z_c])

# Initial model

In [71]:
# import glucifer
# Fig = glucifer.Figure(figsize=(1200,400))
# Fig.Points(surface_tracers.swarm, pointSize=2.0)
# Fig.Points(moho_tracers.swarm, pointSize=2.0)
# #Fig.Points(circles_c.swarm, pointSize=1.5)
# Fig.Points(Model.swarm, Model.materialField, fn_size=2.0)
# Fig.show()


In [72]:
# import numpy as np
# x2 = np.linspace(Model.minCoord[0], Model.maxCoord[0], 3000) * u.kilometer
# y2 = -80. * u.kilometer
# level_tracers = Model.add_passive_tracers(name="Level", vertices=[x2,y2])
# level_tracers2 = Model.level_tracers.add_tracked_field(Model.strainRateField, name="Lev_StrainRate", dataType="float", units=u.sec, count=1, overwrite=True)



# Surface Processes / BADLANDS

In [73]:
# Model.surfaceProcesses = GEO.surfaceProcesses.Badlands(airIndex=[air.index], sedimentIndex=sediment.index,
#                                            XML="ressources/badlands.xml", resolution=1. * u.kilometer, 
#                                            checkpoint_interval=0.01 * u.megayears)


#Model.surfaceProcesses = GEO.surfaceProcesses.SedimentationThreshold(Model.swarm, Model.materialField, air=[air, stickyAir], sediment=[sediment], threshold=0.)

In [74]:
GEO.rcParams["solver"] = "mg" # This is the default so that line is technically not needed.

In [75]:
Model.init_model()

In [76]:
#Fig = Model.plot.temperature(script=["rotate z 30", "rotate x -60"], figsize=(900,600))

In [77]:
#Fig = Model.plot.pressureField(script=["rotate z 30", "rotate x -60"], figsize=(900,600))

In [78]:
mohoT = Model.temperature.evaluate(moho_tracers.swarm)[:,0].mean()
#print("Moho Temperature = ", GEO.Dimensionalize(mohoT, u.degC)) 

In [ ]:
Model.run_for(20 * u.megayear, checkpoint_interval=1 * u.megayear)


Step:1 Model Time:  1.35378691223e-08 megayear dt: 1.35378691223e-08 megayear (2018-07-02 05:08:58)
Step:2 Model Time:  2.8565973209e-08 megayear dt: 1.50281040867e-08 megayear (2018-07-02 05:10:12)
Step:3 Model Time:  4.32556737829e-08 megayear dt: 1.46897005739e-08 megayear (2018-07-02 05:11:30)
Step:4 Model Time:  5.77318818862e-08 megayear dt: 1.44762081034e-08 megayear (2018-07-02 05:12:48)
Step:5 Model Time:  7.20634134746e-08 megayear dt: 1.43315315884e-08 megayear (2018-07-02 05:14:06)
Step:6 Model Time:  8.62901493942e-08 megayear dt: 1.42267359196e-08 megayear (2018-07-02 05:15:23)
Step:7 Model Time:  1.00434558338e-07 megayear dt: 1.41444089433e-08 megayear (2018-07-02 05:16:39)
Step:8 Model Time:  1.14519872412e-07 megayear dt: 1.4085314074e-08 megayear (2018-07-02 05:17:51)
Step:9 Model Time:  1.28565406459e-07 megayear dt: 1.40455340471e-08 megayear (2018-07-02 05:19:03)
Step:10 Model Time:  1.42581910923e-07 megayear dt: 1.4016504464e-08 megayear (2018-07-02 05:20:14)
St